In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

import wandb

In [2]:
# Constants
H_tower = 80
H_HR = 8
W_HR = 7

import pandas as pd
data = pd.read_excel('../data/附件.xlsx', sheet_name='Sheet1')
initial_position = data.values


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
# 常数计算
## DNI
def cal_DNI(H,alpha):
    """
    H: 海拔高度, km
    """
    a = 0.4237 - 0.00821 * (6 - H ) ** 2
    b = 0.5055 + 0.00595 * (6.5 - H ) ** 2
    c = 0.2711 + 0.01858 * (2.5 - H ) ** 2
    G0 = 1.366 # kW/m^2
    DNI = G0 * (a+b*np.exp(-c/np.sin(alpha)))
    return DNI

def cal_sunlight_angle(phi,H):
    """
    计算太阳高度角
    phi: 纬度
    """
    phi = phi * np.pi / 180
    omega_list = [-torch.pi/4,
                  -torch.pi/8,
                  0,
                  torch.pi/8,
                  torch.pi/4]
    omega_list = torch.tensor(omega_list).unsqueeze(0)
    month_list = range(3,15)
    month_list = torch.tensor(month_list).unsqueeze(1)
    input = torch.zeros((12*5,3))
    sin_delta = torch.sin(2*np.pi*(30*(month_list-3))/365)*np.sin(2*np.pi/365*23.45)
    cos_delta = torch.sqrt(1-sin_delta**2)
    # sin_alpha is a tensor of size (len(month_list),len(omega_list))
    sin_alpha = np.sin(phi) * sin_delta + np.cos(phi) * cos_delta * torch.cos(omega_list)
    cos_alpha = torch.sqrt(1 - sin_alpha ** 2)
    # cos_gamma is a tensor of size (len(month_list),len(omega_list))
    cos_gamma = (sin_delta - sin_alpha * np.sin(phi)) / (np.cos(phi) * cos_alpha)
    alpha = torch.atan(sin_alpha / cos_alpha) # is a tensor of size (12,5)
    gamma = torch.atan(torch.sqrt(1 - cos_gamma ** 2) / cos_gamma) # is a tensor of size (12,5)

    DNI = cal_DNI(H,alpha) # is a tensor of size (12,5)
    # to (len(month_list)*len(omega_list),3)
    input[:,0] = DNI.view(-1)
    input[:,1] = alpha.view(-1)
    input[:,2] = gamma.view(-1)

    # print('alpha',alpha)
    # print('sin_alpha',sin_alpha)
    return input.to(device)
    

In [10]:
class reflect_matrix(torch.nn.Module):
    def __init__(self, num_panel, trainning_dict, initial_posit, initial_areas, initial_heights):
        """
        num_panel: number of panels
        parameters: the coordinates of the reflect matrix, (x, y)
        areas: the areas of each panel, can be different, paremeters: height, width - (num_panel, 2)
        heights: the heights of each panel, can be different
        """
        super(reflect_matrix, self).__init__()
        self.num_panel = num_panel
        self.x = torch.nn.Parameter(torch.tensor(initial_posit[:, 0]).unsqueeze(0))
        self.y = torch.nn.Parameter(torch.tensor(initial_posit[:, 1]).unsqueeze(0))

        self.H = torch.nn.Parameter(torch.tensor(initial_areas[:, 0]).unsqueeze(0))
        self.W = torch.nn.Parameter(torch.tensor(initial_areas[:, 1]).unsqueeze(0))

        self.heights = torch.nn.Parameter(torch.tensor((initial_heights)).unsqueeze(0))

        for name, value in trainning_dict.items():
            if name == 'x':
                self.x.requires_grad = value
            elif name == 'y':
                self.y.requires_grad = value
            elif name == 'H':
                self.H.requires_grad = value
            elif name == 'W':
                self.W.requires_grad = value
            elif name == 'heights':
                self.heights.requires_grad = value
            else:
                print('Wrong name!'+name)
            
        self.d_HR = torch.sqrt((self.x) ** 2 + (self.y) ** 2 + (H_tower - H_HR) ** 2).unsqueeze(0)
        # create a list of neighbour for each panel
        self.neighbour_mask = None

        self.d_HR = self.d_HR.detach().to(device)
        
        self.find_neighbour()
        print('------------------')
        print('Shapes of parameters:')
        print('x:', self.x.shape, 'y:', self.y.shape)
        print('H:', self.H.shape, 'W:', self.W.shape)
        print('heights:', self.heights.shape)
        print('d_HR:', self.d_HR.shape)
        print('neighbour_mask:', self.neighbour_mask.shape)
        print('------------------')


    def cal_efficency(self, input, lower_limit = 6*1e5):
        """
        input: the input contains the information of sun light, specifically
        - data_γ: 太阳光方位角 gamma_s
        - data_α: 太阳光高度角 alpha_s
        - data_dni: DNI
        """
        data_γ = input[:,0].unsqueeze(1)
        data_α = input[:,1].unsqueeze(1)
        data_dni = input[:,2].unsqueeze(1)
        

        def cal_eta_sb(eta_cos):
            # TODO: calculate the eta_sb
            ## only calculate neighbour
            ### calculate the vector from panel a to panel b, `vector_ab = posit_b - posit_a`, is a tensor of size (num_panel, num_panel, 3)
            vector_ab = torch.cat((self.x.unsqueeze(2) - self.x.T.unsqueeze(2), \
                                      self.y.unsqueeze(2) - self.y.T.unsqueeze(2), \
                                      self.heights.unsqueeze(2) - self.heights.T.unsqueeze(2)), dim = 2) # is a tensor of size (num_panel, num_panel, 3)
            #### mask
            vector_ab = vector_ab * self.neighbour_mask.unsqueeze(2)
            #### calculate the distance between two panels
            distance_sq = torch.sum(vector_ab ** 2, dim = 2) # is a tensor of size (num_panel, num_panel)

            ray_in_direction = torch.cat((torch.cos(data_α) * torch.sin(data_γ), \
                                            torch.cos(data_α) * torch.cos(data_γ), \
                                            torch.sin(data_α)), dim = 1).unsqueeze(1) # is a tensor of size (num_case, 3)

            ray_out_direction = torch.cat((-self.x.unsqueeze(2), \
                                            -self.y.unsqueeze(2), \
                                            (H_tower - self.heights).unsqueeze(2)), dim = 2) # is a tensor of size (1,num_panel, 3)
            #### calculate the approximate diagonal of the panel
            diagonal_approx = torch.sqrt(self.H ** 2 + self.H ** 2*eta_cos ** 2).squeeze(0) # is a tensor of size (num_panel, )
            Area_approx = self.H * self.H * eta_cos # is a tensor of size (num_panel, )

            #### calculate the projection of the vector_ab on the ray_in_direction, the output is a tensor of size (num_case, num_panel, num_panel); 
            # (num_case, 1, 1, 3) * (1, num_panel, num_panel, 3) = (num_case, num_panel, num_panel)
            vector_ab_projection = torch.sqrt(distance_sq.unsqueeze(0)-\
                                              torch.sum(ray_in_direction.unsqueeze(2)*\
                                                        vector_ab.unsqueeze(0),dim=-1) ** 2) # is a tensor of size (num_case, num_panel, num_panel)

            D_PQ = 0.5*(diagonal_approx.unsqueeze(2) + diagonal_approx.unsqueeze(1)) - vector_ab_projection*(self.neighbour_mask.unsqueeze(0))
            D_PQ[D_PQ < 0] = 0 # is a tensor of size (num_panel, num_panel)
            shade_area_in = torch.sum(D_PQ**2, dim = 2) * Area_approx / diagonal_approx ** 2 # is a tensor of size (num_case, num_panel)
            shade_area_in = shade_area_in.squeeze(0)

            #### calculate the projection of the vector_ab on the ray_out_direction, the output is a tensor of size (num_case, num_panel, num_panel); 
            # (1, num_panel, 1, 3) * (1, num_panel, num_panel, 3) = (num_case, num_panel, num_panel)
            vector_ab_projection = torch.sqrt(distance_sq.unsqueeze(0)-\
                                                torch.sum(ray_out_direction.unsqueeze(2)*\
                                                            vector_ab.unsqueeze(0),dim=-1) ** 2) # is a tensor of size (num_case, num_panel, num_panel)
            D_PQ = 0.5*(diagonal_approx.unsqueeze(2) + diagonal_approx.unsqueeze(1)) - vector_ab_projection*(self.neighbour_mask.unsqueeze(0))
            D_PQ[D_PQ < 0] = 0 # is a tensor of size (num_panel, num_panel)
            shade_area_out = torch.sum(D_PQ**2, dim = 2) * Area_approx / diagonal_approx ** 2 # is a tensor of size (num_case, num_panel)
            shade_area_out = shade_area_out.squeeze(0)
            
            #### calculate the eta_sb
            eta_sb = 1-shade_area_in-shade_area_out
            return eta_sb # is a tensor of size (num_case, num_panel)

        def cal_eta_cos():
            # $$cos\theta = \sqrt{\frac{1}{2}[1+\frac{1}{d_{HR}}(-xcos\alpha_ssin\gamma_s-ycos\alpha_s\cos\gamma_s+sin\alpha_s(h_1-h_2))]}$$
            A = self.x * torch.cos(data_α) * torch.sin(data_γ)
            B = self.y * torch.cos(data_α) * torch.cos(data_γ)
            C = torch.sin(data_α) * (self.heights - H_HR)
            eta_cos = torch.sqrt((1 + (1 / self.d_HR) * (- A - \
                                B + \
                                C)) / 2)
            # return size is (num_case, num_panel)
            return eta_cos # is a tensor of size (num_case, num_panel)
        
        def cal_eta_at():
            #calculate the eta_at
            eta_at = 0.99321 - 1.117e-4 * self.d_HR + 1.97e-8 * self.d_HR ** 2
            return eta_at
        
        def cal_eta_trunc():
            #calculate the eta_trunc
            ## beta = arctan((H_tower - H_HR) / d_HR)
            beta = torch.atan((H_tower - H_HR) / self.d_HR)
            ## cos_theta_v = \sqrt{\frac{1+cos2\theta_v}{2}}=\sqrt{\frac{1+cos(\alpha_s-\beta)}{2}}
            cos_theta_v = torch.sqrt((1 + torch.cos(2 * (data_α.unsqueeze(1) - beta))) / 2)
            ## cos_theta_h = =\sqrt{\frac{1+cos2\theta_h}{2}}=-\frac{1}{\sqrt{x^2+y^2}}(xsin\gamma_s+ycos\gamma_s)
            cos_theta_h = torch.sqrt((1+(self.x * torch.sin(data_γ) + self.y * \
                             torch.cos(data_γ))/self.d_HR)/2)
            print('cos_theta_v:', cos_theta_v)
            print('cos_theta_h:', cos_theta_h)
            ## H_ratio = min(1, \frac{H_HR}{H_ref}*cos\theta_v/cos\beta)
            H_ratio = torch.min(torch.tensor(1), H_HR / (self.H * cos_theta_v / torch.cos(beta)))
            ## W_ratio = min(1, \frac{W_HR}{W_ref}*cos\theta_h)
            W_ratio = torch.min(torch.tensor(1), W_HR / (self.W * cos_theta_h))
            ## A_ratio = H_ratio * W_ratio
            A_ratio = H_ratio * W_ratio
            return A_ratio
        
        def cal_eta_ref():
            #calculate the eta_ref
            return 0.92
        
        
        
        # calculate eta
        eta_cos = cal_eta_cos()
        eta_at = cal_eta_at()
        eta_trunc = cal_eta_trunc()
        eta_ref = cal_eta_ref()
        eta_sb = cal_eta_sb(eta_cos)                                          

        eta = eta_sb * eta_cos * eta_at * eta_trunc * eta_ref # each is a tensor of size (num_panel, )
        print('eta:', eta.shape)
        print('eta_cos:', eta_cos)
        print('eta_at:', eta_at)
        print('eta_trunc:', eta_trunc)
        print('eta_ref:', eta_ref)
        print('eta_sb:', eta_sb)


        # calculate the efficiency per unit area
        E_field = torch.sum(data_dni * eta)

        # penalty
        ## the sum of the efficiency should be larger than 0.5
        # lower_limit = lower_limit
        lower_limit_penalty = torch.min(torch.tensor(0), E_field - lower_limit)

        penalty = lower_limit_penalty*0.1
        return E_field, penalty

    def find_neighbour(self):
        """
        find the neighbour of each panel
        """
        search_range_sq = 2 * self.heights**2
        # if the distance between two panels is smaller than the search range, then they are neighbours
        # the distance between two panels
        distance_sq = (self.x - self.x.T) ** 2 + (self.y - self.y.T) ** 2
        # find the neighbour
        self.neighbour_mask = (distance_sq < search_range_sq).float()
        # the mask has no effect on the panel itself
        self.neighbour_mask[torch.arange(self.num_panel), torch.arange(self.num_panel)] = 0
        # no gradient
        self.neighbour_mask = self.neighbour_mask.detach().to(device)
        return None
    
    def forward(self, input):
        E_field, penalty = self.cal_efficency(input)
        loss = -E_field + penalty
        return loss

In [11]:
trainning_dict = {
    'x': True,
    'y': True,
    'H': False,
    'W': False,
    'heights': False
}

num_panel = initial_position.shape[0]
H_ref, W_ref = 6, 6
H_ref_base = 4

initial_posit = initial_position
initial_areas = np.hstack((np.ones((num_panel, 1)) * H_ref, np.ones((num_panel, 1)) * W_ref))
initial_heights = np.ones(num_panel) * H_ref_base

input =  cal_sunlight_angle(39.4, 3000)
print('Shape of input:', input.shape)

matrix = reflect_matrix(num_panel, trainning_dict, initial_posit, initial_areas, initial_heights).to(device)
E_field, penalty = matrix.cal_efficency(input)

print('Before trainning:')
print('E_field:', E_field)
print('penalty:', penalty)

# TODO: train the reflect_matrix
## maximize the E_field
loss = - E_field + penalty

## optimize
### set trainning hyperparameters
optimizer = torch.optim.Adam(matrix.parameters(), lr = 0.01)
num_epoch = 100
### print trainning information
print('trainning information:')
print('trainning_dict: {}'.format(trainning_dict))
print('num_epoch: {}'.format(num_epoch))
print('optimizer: {}'.format(optimizer))


for epoch in range(num_epoch):
    optimizer.zero_grad()
    matrix.find_neighbour()
    E_field, penalty = matrix.cal_efficency(input)
    loss = - E_field + penalty
    loss.backward()
    optimizer.step()
    print('epoch: {}, loss: {}, E_field: {}, penalty: {}'.format(epoch, loss, E_field, penalty))




Shape of input: torch.Size([60, 3])
------------------
Shapes of parameters:
x: torch.Size([1, 1745]) y: torch.Size([1, 1745])
H: torch.Size([1, 1745]) W: torch.Size([1, 1745])
heights: torch.Size([1, 1745])
d_HR: torch.Size([1, 1, 1745])
neighbour_mask: torch.Size([1745, 1745])
------------------
cos_theta_v: 

tensor([[[0.9975, 0.9975, 0.9975,  ..., 0.9315, 0.9315, 0.9315]],

        [[0.9587, 0.9587, 0.9587,  ..., 0.8314, 0.8314, 0.8314]],

        [[0.9300, 0.9300, 0.9300,  ..., 0.7793, 0.7793, 0.7793]],

        ...,

        [[0.9885, 0.9885, 0.9885,  ..., 0.8992, 0.8992, 0.8992]],

        [[0.9967, 0.9967, 0.9967,  ..., 0.9279, 0.9279, 0.9279]],

        [[0.9945, 0.9945, 0.9945,  ..., 0.9808, 0.9808, 0.9808]]],
       device='cuda:0', dtype=torch.float64)
cos_theta_h: tensor([[[0.9450, 0.9515, 0.9555,  ..., 0.9557, 0.9621, 0.9679],
         [0.9450, 0.9515, 0.9555,  ..., 0.9557, 0.9621, 0.9679],
         [0.9450, 0.9515, 0.9555,  ..., 0.9557, 0.9621, 0.9679],
         ...,
         [0.9450, 0.9515, 0.9555,  ..., 0.9557, 0.9621, 0.9679],
         [0.9450, 0.9515, 0.9555,  ..., 0.9557, 0.9621, 0.9679],
         [0.9450, 0.9515, 0.9555,  ..., 0.9557, 0.9621, 0.9679]]],
       device='cuda:0', dtype=torch.float64, grad_fn=<SqrtBackward0>)
eta: torch.Size([60, 60, 1745])
eta_cos: tensor([[

KeyboardInterrupt: 